In [1]:
import os,sys

from keras.models import Model
from keras.layers import Input,LSTM,GRU,Dense,Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [2]:
BATCH_SIZE = 64
EPOCHS = 20
LSTM_NODES =256
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 100

In [4]:
input_sentences = []
output_sentences = []
output_sentences_inputs = []

count = 0
for line in open('./ben.txt'):
    count += 1

    if count > NUM_SENTENCES:
        break

    if '\t' not in line:
        continue

    spLine = line.rstrip().split('\t')
    input_sentence , output = spLine[0] , spLine[1]
    output_sentence = output + ' <eos>'
    output_sentences_input = '<sos> ' + output

    input_sentences.append(input_sentence)
    output_sentences.append(output_sentence)
    output_sentences_inputs.append(output_sentences_input)

print("num samples input: ",len(input_sentences))
print("num samples output: ",len(output_sentences))
print("num samples input output: ",len(output_sentences_inputs))


num samples input:  4619
num samples output:  4619
num samples input output:  4619


In [5]:
for i in range(5):
    x = np.random.randint(0,4349)
    print(input_sentences[x])
    print(output_sentences[x])
    print(output_sentences_inputs[x])

I'm getting off at the next station.
আমি পরের স্টেশনে নেমে যাচ্ছি। <eos>
<sos> আমি পরের স্টেশনে নেমে যাচ্ছি।
What fun!
কি মজা! <eos>
<sos> কি মজা!
I don't want Tom to find me.
আমি চাই না টম আমাকে খুঁজে পাক। <eos>
<sos> আমি চাই না টম আমাকে খুঁজে পাক।
Keep quiet.
চুপ করো। <eos>
<sos> চুপ করো।
Forgive Tom.
টমকে ক্ষমা করে দাও। <eos>
<sos> টমকে ক্ষমা করে দাও।


In [6]:
input_tokenizer = Tokenizer(num_words = MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print('Total Unique Words: ',len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print('Length of Longest Sequence: ',max_input_len)

Total Unique Words:  1902
Length of Longest Sequence:  20


In [7]:
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the output: 3642
Length of longest sentence in the output: 19


In [8]:
encoder_input_sequences = pad_sequences(input_integer_seq,maxlen=max_input_len)

print('Input Encoder Shape: ',encoder_input_sequences.shape)
print(encoder_input_sequences[10])

Input Encoder Shape:  (4619, 20)
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 71]


In [9]:
print(word2idx_inputs['i'])
print(word2idx_inputs['will'])

1
70


In [10]:
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences.shape:", decoder_input_sequences.shape)
print("decoder_input_sequences[172]:", decoder_input_sequences[172])

decoder_input_sequences.shape: (4619, 19)
decoder_input_sequences[172]: [   2  445 1177    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


In [11]:
print(word2idx_outputs['<sos>'])
print(word2idx_outputs['"আপনি'])
print(word2idx_outputs['একজন'])

2
1148
86


In [44]:
embeddings_dictionary = dict()

glove_file = open('./glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [51]:
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_SIZE))
for word, index in word2idx_inputs.items():
    embedding_vector = embeddings_dictionary.get(word)

    if  embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [52]:
print(embeddings_dictionary['yet'])

[-0.11941    0.54188    0.62174   -0.10935   -0.15776   -0.013474
  0.23946   -0.044287   0.37478   -0.049821   0.11609    0.24012
  0.53843   -0.4148    -0.078387  -0.20494   -0.028806   0.42345
 -0.31811    0.1978     0.34817   -0.088881  -0.026402  -0.31054
 -0.08553   -0.056577  -0.051455  -0.37321   -0.20699   -0.42797
  0.047632   0.1657    -0.16632   -0.045508   0.32539    0.31902
 -0.17114   -0.24263   -0.32311   -0.30039   -0.3664    -0.10475
  0.27917   -0.018142  -0.065002  -0.13421    0.75376   -0.15472
  0.13552   -0.83338    0.52732   -0.33982    0.47992    0.9676
  0.20516   -2.4215    -0.36029   -0.092199   1.4472     0.51091
 -0.19357    0.79376   -0.42247   -0.21033    0.98025   -0.27021
  0.29239    0.56319   -0.28621   -0.45666    0.32348   -0.22252
 -0.29755   -0.29786    0.34122   -0.0089719 -0.40418   -0.39275
 -0.87327   -0.29921    0.62069   -0.13045   -0.34651   -0.11863
 -1.6367    -0.49879    0.36034    0.042055  -0.64994   -0.26185
  0.16331   -0.090286  -0

In [53]:
print(embedding_matrix[184])

[-0.11941     0.54188001  0.62173998 -0.10935    -0.15775999 -0.013474
  0.23946001 -0.044287    0.37478    -0.049821    0.11609     0.24011999
  0.53842998 -0.41479999 -0.078387   -0.20494001 -0.028806    0.42344999
 -0.31810999  0.1978      0.34817001 -0.088881   -0.026402   -0.31053999
 -0.08553    -0.056577   -0.051455   -0.37321001 -0.20699    -0.42796999
  0.047632    0.1657     -0.16632    -0.045508    0.32539001  0.31902
 -0.17114    -0.24263    -0.32311001 -0.30039001 -0.3664     -0.10475
  0.27917001 -0.018142   -0.065002   -0.13421001  0.75375998 -0.15471999
  0.13552    -0.83337998  0.52732003 -0.33982     0.47992     0.96759999
  0.20516001 -2.42149997 -0.36028999 -0.092199    1.44719994  0.51090997
 -0.19357     0.79376    -0.42247    -0.21032999  0.98025    -0.27021
  0.29238999  0.56318998 -0.28621    -0.45666     0.32348001 -0.22251999
 -0.29754999 -0.29786     0.34121999 -0.0089719  -0.40417999 -0.39274999
 -0.87326998 -0.29921001  0.62068999 -0.13045    -0.34650999 -

In [54]:
embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_input_len)

In [55]:
decoder_targets_one_hot = np.zeros((
        len(input_sentences),
        max_out_len,
        num_words_output
    ),
    dtype='float32'
)

In [56]:
decoder_targets_one_hot.shape

(4619, 19, 3643)

In [57]:
for i, d in enumerate(decoder_input_sequences):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

In [58]:
encoder_inputs_placeholder = Input(shape=(max_input_len,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

2022-08-20 13:02:06.973495: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-20 13:02:07.149596: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-20 13:02:07.150305: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-20 13:02:07.158660: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [59]:
decoder_inputs_placeholder = Input(shape=(max_out_len,))

decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)

In [60]:
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [61]:
model = Model([encoder_inputs_placeholder,
  decoder_inputs_placeholder], decoder_outputs)
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [66]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [68]:
model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
)

Epoch 1/20


2022-08-20 13:10:07.095205: E tensorflow/stream_executor/cuda/cuda_dnn.cc:361] Loaded runtime CuDNN library: 8.0.5 but source was compiled with: 8.1.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2022-08-20 13:10:07.096771: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at cudnn_rnn_ops.cc:1556 : UNKNOWN: Fail to find the dnn implementation.
2022-08-20 13:10:14.819208: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 16.90MiB (rounded to 17719552)requested by op SameWorkerRecvDone
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summa

UnknownError: Graph execution error:

Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[model/lstm/PartitionedCall]] [Op:__inference_train_function_5790]